# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui

sinasc.isna().sum()



ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui 
#['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
# 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
#sinasc.columns
sinasc.drop(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 
       'CODOCUPMAE', 'QTDFILMORT', 'CODMUNRES',
       'PARTO', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1','RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],axis = 1, inplace = True)
print(sinasc.columns)



Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5'],
      dtype='object')


In [4]:
# 4) seu código aqui
print("Número total de linhas do DataFrame: " + str(len(sinasc)))
print("Número de linhas a serem eliminadas: " + str(sinasc['APGAR5'].isna().sum()))
#tentei utilizar o dropna para apagar as linhas mas não funcionou, desta forma eu contornei o problema
#sinasc['APGAR5'].dropna(inplace=True)
#sinasc= sinasc['APGAR5'].dropna()
sinasc = sinasc[~sinasc['APGAR5'].isna()]
print("Número de linhas restantes: " + str(len(sinasc)))


Número total de linhas do DataFrame: 27028
Número de linhas a serem eliminadas: 103
Número de linhas restantes: 26925


In [5]:
# 5) seu código aqui
#sinasc.shape ['ESTCIVMAE', 'CONSULTAS']
print(sinasc['ESTCIVMAE'].isna().sum())
sinasc['ESTCIVMAE'].fillna(9,inplace=True)
print(sinasc['ESTCIVMAE'].isna().sum())

print(sinasc['CONSULTAS'].isna().sum())
#para consulta não foi necessário fazer a mudança de "Na" para 9, não existe casos.
#sinasc['CONSULTAS'].fillna(9,inplace=True)
#print(sinasc['CONSULTAS'].isna().sum())

315
0
0


In [6]:
# 6) Seu código aqui
#QTDFILVIVO
print(sinasc['QTDFILVIVO'].isna().sum())
sinasc['QTDFILVIVO'].fillna(0, inplace=True)
print(sinasc['QTDFILVIVO'].isna().sum())

1566
0


In [7]:
# 7) seu código aqui
print(sinasc.isna().sum())
#print(sinasc['ESCMAE'].value_counts())
#Para escolaridade da mãe no momento a decisão é colocar ignorado já que isto foi previsto
#na concepção do arquivo e não está sendo utilizado
sinasc['ESCMAE'].fillna('Ignorado', inplace=True)


#Gestacao
#print(sinasc['GESTACAO'].value_counts())
# Neste caso poderiamos colocar os valores para a opção que ocorre com mais frequência e que é mais normal de acontecer, 
#37 a 41 semanas, mas como temos a opção ignorado, prefiro utiliza-la.
sinasc['GESTACAO'].fillna('Ignorado', inplace=True)


#GRAVIDEZ
#print(sinasc['GRAVIDEZ'].value_counts())
# o Mesmo vale para este caso, como foi previsto o ignorado, vamos utilizá-lo
sinasc['GRAVIDEZ'].fillna('Ignorado', inplace=True)

print(sinasc.isna().sum())

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64
LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64


In [8]:
# 8) seu código aqui

categoria=[]
categoria=sinasc['APGAR5'].map({0:'asfixia severa',1:'asfixia severa',2:'asfixia severa',3:'asfixia severa',
                4:'asfixia moderada',5:'asfixia moderada',6:'asfixia leve',7:'asfixia leve',
                8:'normal', 9:'normal', 10:'normal'}) 
sinasc['apgar5_cat']=categoria
print(sinasc)

       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5 apgar5_cat  
0      37 a 41 semanas    Única          4    10.0     normal  
1      37 a 41 semanas    Única          4     9.0     

In [9]:
# 9) seu código aqui
print(sinasc.columns)
sinasc.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao',
       'gravidez', 'consultas', 'apgar_5', 'apgar_5_cat']
print(sinasc.columns)

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'apgar5_cat'],
      dtype='object')
Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar_5', 'apgar_5_cat'],
      dtype='object')


In [10]:
print(sinasc)

       loc_nasc  idade_mae  est_civ_mae          esc_mae  qtd_fil_vivo  \
0             1         19          5.0      8 a 11 anos           0.0   
1             1         29          2.0      8 a 11 anos           1.0   
2             1         37          9.0      8 a 11 anos           2.0   
3             1         30          5.0  12 anos ou mais           0.0   
4             1         30          2.0      8 a 11 anos           1.0   
...         ...        ...          ...              ...           ...   
27023         1         32          2.0  12 anos ou mais           1.0   
27024         1         19          1.0      8 a 11 anos           0.0   
27025         1         24          2.0      8 a 11 anos           0.0   
27026         1         21          2.0      8 a 11 anos           1.0   
27027         1         18          5.0      8 a 11 anos           1.0   

              gestacao gravidez  consultas  apgar_5 apgar_5_cat  
0      37 a 41 semanas    Única          4   